# SCORE DE PROPENSION ET APPARIEMENT

Cette application s'inspire de l'article de Douglas Almond, Kenneth Y. Chay, and David S. Lee : "The Costs of Low Birth Weight", *Quarterly Journal of Economics*, 120(3):1031-1083.

L'objectif : étudier l'effet causal du fait que les mères fument durant leur grossesse sur le poids des enfants à la naissance. Connaître les déterminants du poids à la naissance hors facteurs génétiques est crucial car le poids à la naissance a une importante répercussion sur une série de variables en santé, développement, éducation, et *in fine* revenu des enfants.

Les données sont issues du registre des naissances de Pennsylvanie en 1993. Chaque ligne rapporte des observations sur la mère et son enfant. Le codebook est disponible : **TD7_codebook.txt**.

## 1. Intuition et modèle linéaire

Dans ce contexte, quelle est notre variable d'intérêt ou résultat ? Notre variable de traitement ? Identifiez ces variables dans le codebook : **TD7_codebook.txt**.

Ouvrir les données **TD7_ps1_postdrop.dta**. 

On souhaite travailler sur le sous échantillon des femmes de 32 ans. Créer le sous-échantillon que l'on nommera d. 

Estimer le modèle simple. Quelle interprétation donner des résultats ? Doit-on interdire aux femmes enceintes de fumer au vu de ces résultats ?

Supposons que le fait pour une mère de fumer ou non est assigné de manière aléatoire, conditionnellement à d'autres variables prédéterminées qui déterminent le poids de l'enfant à la naissance. 

a) Quelle méthode économétrique utiliseriez-vous ? 

b) Quelles variables de contrôle choisiriez-vous ?

c) Quelle est la source de mauvaise spécification du modèle si on choisit cette forme ? 

Estimer le modèle multiple avec les variables prédéterminées : *pldel3* *ormoth* *mrace3* *dmeduc* *dmar* *dfage* *orfath* *dfeduc*.

## 2. Création du score de propension

Décrivez en quelques lignes en quoi consiste le score de propension et comment il contribue à estimer l'effet moyen causal du traitement (ici : le fait de fumer durant la grossesse) quand le traitement est assigné de manière aléatoire une fois conditionné par les variables prédéterminées.

Décrivez en quelques lignes comment s'obtient le score de propension que nous notons $\hat{p}_{i}$ dans le modèle et que nous nommons *pr_score* dans les données.

On estime le score de propension : 

In [ ]:
propensity <- glm(tobacco ~  dmage+ dmeduc+ dmar
                     +dfage +dfeduc+ phyper +pre4000+ alcohol +wgain, binomial(probit), data=d)
summary(propensity)

Etes-vous d'accord avec cette estimation ? 

Estimez à nouveau le score de propension avec les variables prédéterminées : *pldel3* *ormoth* *mrace3* *dmeduc* *dmar* *dfage* *orfath* *dfeduc*

On souhaite montrer graphiquement la distribution du score de propension selon le groupe (unités traitées, unités de contrôle -- ici mères fumeuses ou non-fumeuses), ainsi qu'un tableau donnant les statistiques descriptives de ce score par groupe. Pour cela on doit d'abord prédire le score :

Examinez les 6 premières lignes de ce dataframe : 

On adopte deux types de représentations graphiques du support commun : 
- un hisogramme séparé, 
- ou une estimation empirique de la densité par fonction noyau

Les deux graphiques donnent la même information sur la qualité du support commun des scores estimés) :

In [ ]:
##histogramme :
library(plyr)
library(dplyr)
library(ggplot2)
labs <- paste("Statut:", c("Fumeur", "Non-fumeur"))
prs_df %>%
  mutate(tobacco = ifelse(tobacco == 1, labs[1], labs[2])) %>%
  ggplot(aes(x = pr_score)) +
  geom_histogram(color = "white", bins=100) +
  facet_wrap(~tobacco) +
  xlab("Score de propension: probabilité de fumer pendant la grossesse") +
  ylab("") +
  theme_bw()

In [ ]:
#ou encore densité empirique : 
plot(density(prs_df$pr_score[
  prs_df$tobacco==0]),col="blue",lty="dotted",main="Densité noyau du score de propension",ylab="Densité",xlab="Score")
lines(density(prs_df$pr_score[
  prs_df$tobacco==1]), col="red")
legend("topright", c("Non-fumeurs","Fumeurs"), lty = c(3,1), col = c("blue","red"))

Qu'en conclure ? 

Que faire pour améliorer la qualité du support commun ?

## 3. Estimations avec score de propension

Nous utilisons $\hat{p}_{i}$ pour améliorer l'estimation de l'effet causal du fait de fumer sur le poids des nouveaux-nés. Commencer par ajouter le score *pr_score* au dataframe *d* et trimmer les données.

### 3.1 Utilisation directe du score de propension

Nous introduisons d'abord $\hat{p}_{i}$ comme variable dans l'équation simple: $dbrwt_{i} = \gamma D_{i} + u_{i}$. Estimer. Quel est l'effet moyen du traitement ? Commenter.

Pourquoi l'introduction du score de propension n'a-t-elle que peu de valeur ajoutée pour résoudre notre problème de sélection sur observables par rapport à un modèle linéaire classique ?  

In [ ]:
#pour rappel:
coef(simple)[2]
coef(multiple)[2]
coef(simple_p)[2]

### 3.2 Pondération par le score de propension

Nous utilisons la méthode proposée par Hirano, Imbens et Ridder (2003) de pondération par score de propension.

a) Donner la formule de la pondération.

b) Expliquer en quoi cette pondération permet de mieux mesurer l'effet moyen du traitement. 

Créer le poids. 

Pour introduire les poids dans la régression, on utilise l'option **weights=** dans **lm()**. 
Estimer le modèle. Quel est l'effet moyen du traitement ?

Quel est le principal avantage de cette approche ? Le principal désavantage ? 

### 3.3 Estimation par bloc sur le score de propension

Nous utilisons la méthode proposée par Dehejia et Wahba (2002). 

a) Comment fonctionne la stratification ? Pourquoi le support commun est-il particulièrement crucial ? 

b) On calcule l'effet moyen du traitement grâce à 100 blocs. On obtient : $\hat{\tau}=-213,62$. Expliquez les différentes étapes du calcul. 

c) Que faudrait-il vérifier au sein de chaque bloc ? 


### 3.4 Appariement par le score de propension

Le score de propension peut être mobilisé pour construire un groupe de contrôle qui ressemble au groupe de traitement : ce sont les techniques d'appariement par score de propension. 

Citez une méthode simple d'appariement par score de propension.

On souhaite calculer l'effet moyen du traitement avec cette méthode. A combien d'appariement va-t-on procéder ?

On estime le modèle avec appariement par le plus proche voisin (sur un sous-échantillon des fumeuses de 32 ans). Pour cela :
- on charge le package "MatchIt" (et s'il le faut on l'installe d'abord)
- on cherche les plus proches voisins avec la fonction matchit(y~x, method="nearest", data=...), où y est le traitement, et x le score
- on génère le dataframe qui inclut les observations traitées et leur plus proche voisin
- on analyse la qualité de l'appariement


Chargement :

Recherche des plus proches voisins : 

Constitution du dataframe :

Qualité de l'appariement : refaire le graphique de votre choix de la partie 2 (histogramme ou densité), avec le dataframe crée ci-dessus.

In [ ]:
#NOTE : MatchIt donne des informations sur la qualité de l'appariemment :
summary(voisins) 
plot(voisins)

Estimer le modèle simple sur les données appariées. 

NOTE : On peut représenter graphiquement les effets mesurés par les différents modèles estimés :

In [ ]:
#modèle simple
est1<-coef(simple)[2]
c1<-confint(simple)[2,]

#multiple
est2<-coef(multiple)[2]
c2<-confint(multiple)[2,]

#modèle simple avec score
est3<-coef(simple_p)[2]
c3<-confint(simple_p)[2,]

#modèle repondéré simple
est4<-coef(reweight)[2]
c4<-confint(reweight)[2,]

#modèle repondéré multiple
est5<-coef(reweight_withcontrols)[2]
c5<-confint(reweight_withcontrols)[2,]

#appariement par le plus plus proche voisin
est6<-coef(nn)[2]
c6<-confint(nn)[2,]

In [ ]:
#création du data frame 
est <- data.frame(est=c(est1,est2,est3,est4,est5,est6),
                    c_inf=c(c1[1],c2[1],c3[1],c4[1],c5[1],c6[1]),
                    c_sup=c(c1[2],c2[2],c3[2],c4[2],c5[2],c6[2]),
                    model=c("Simple","Multiple","Simple+score","Simple+repond","Multiple+pond","Plus proche voisin"))

est

In [ ]:
#graphique
ggplot(est,aes(x=model,y=est))+ geom_point() +
    geom_errorbar(aes(ymin=c_inf,ymax=c_sup),width=0.2) +
    geom_hline(yintercept=0, color="red") +
    xlab("Modèles") +
    ylab("Effet de fumer lors de la grossesse sur le poids à la naissance")


OPTIONNEL : En quoi consiste l'appariement par score de propension sur la base de la fonction noyau ?